In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


 - More routing iterations: Does more routing help classification. (This routing operation is sweet, can we benefit from it more)
 
 - Varying the number of capsules: Does the number(32)/size(8) of the PrimaryCaps layer help classification? (This network is heavy as f!@#)
 
 - More initial convolutional layers: Do smaller filters (3x3, say) with more standard conv layers help classification? (This network is heavy as f!@#)
 
 - No primary capsules: Can we get away with just convolutional layers, routing, DigiCaps and reconstruction?
 

In [4]:
def conv_layer(inputs, filters, kernel_size=9, strides=1, padding='valid', activation=None):
    return tf.layers.conv2d(inputs=inputs, 
                            filters=filters, 
                            kernel_size=kernel_size, 
                            strides=strides, 
                            padding=padding, 
                            activation=activation, 
                            kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False))

In [5]:
BATCH_SIZE = 64
LEARNING_RATE = 0.001
ROUTING_ITERS = 3 

In [68]:
x = np.arange(2*3*2).reshape((2, 3, 2))
y = np.arange(2*3*2).reshape((2, 3, 2)) + 50
M = np.ones(6*3*2).reshape((6, 3, 2))
M = np.tile(M, (2, 1, 1, 1))
b = np.stack([x, y], axis=1).reshape((2, 3*2, 2))
print(b.shape, M.shape)
# print(b)
# np.einsum('ijk,nmpk->ijnmp', b, M)

(2, 6, 2) (2, 6, 3, 2)


In [69]:
n_pc = 4
pc = 2
dc = 3
a = tf.constant(np.arange(1, 1 + n_pc*pc, dtype=np.int32),
                shape=[2, n_pc, pc])
b = tf.constant(np.arange(13, 25, dtype=np.int32),
                shape=[1, n_pc, pc, dc])
b = tf.tile(b, (2, 1, 1, 1))
a = tf.reshape(a, shape=(2, 3, 1, 2)) 
c = tf.matmul(a, b)
c = tf.reshape(c, shape=(2, 3, 2))
with tf.Session() as sess:
    print(sess.run(c))

[[[ 43  46]
  [127 134]
  [243 254]]

 [[211 226]
  [343 362]
  [507 530]]]


In [65]:
/8

6875.0

In [2]:
def caps_net_model_fn(features, labels, mode):
    
    # image layer
    input_layer = tf.reshape(features['x'], [-1, 28, 28, 1])
    
    # save the batch size for later (<= 64)
    batch_size = input_layer.get_shape()[0]
    n_primary_caps = 32
    primary_caps_size = 8
    digi_caps_size = 16
    
    # conv layer (regular convolutional layer)
    with tf.variable_scope('conv_layer'):
        conv1 = conv_layer(input_layer, 256, activation=tf.nn.relu)
        assert conv1.get_shape() == [batch_size, 20, 20, 256]
    
    # first capsule layer (PrimaryCaps)
    capsules = []
    for i in range(32):
        with tf.variable_scope('capsule_1_' + str(i + 1)):
            caps_i = conv_layer(conv1, primary_caps_size, strides=2)
            reshape = tf.reshape(caps_i, shape=(-1, 6*6, primary_caps_size))
            capsules.append(reshape)
    assert capsules[0].get_shape() == [batch_size, 6*6, primary_caps_size]
    
    # stack and reshape
    #
    # here we reshape the capsule outputs (i.e. the 8D vectors)
    # to be 1x8 matrices, this will enable us to use tf.matmul
    # to calculate the inputs (u_hat_ij's) to each DigiCaps
    # capsule in one shot
    capsules = tf.stack(capsules, axis=1)
    capsules = tf.reshape(capsules, shape=(-1, 6*6*n_primary_caps, 1, primary_caps_size))
    assert capsules.get_shape() == [batch_size, 6*6*n_primary_caps, 1, primary_caps_size]
    
    # second capsule layer (DigiCaps)
    u_hat = []
    for j in range(10):
        with tf.variable_scope('->capsule_{}'.format(j)):
            name = 'W_i{}'.format(j)
            weights_to_j = tf.get_variable(name=name, shape=(1, 6*6*n_primary_caps, primary_caps_size, digi_caps_size))
            weights_to_j = tf.tile(weights_to_j, (batch_size, 1, 1, 1))
            u_hat_ji = tf.matmul(capsules, weights_to_j, )
            
    
    
            